**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「2023-2024シーズンのNBA」に関する質問を取り扱います。

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。

### 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 6.40 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [13]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
questions = [
    "NBAって何？",
    "2023-2024シーズンのNBAのウェスタンカンファレンスの最高勝率チームとその戦績は？",
    "有名なNBA選手を5人教えて",
    "2023-2024シーズンのAll NBA First teamに選ばれた5人は？",
    "2024-2025のシーズンのNBAはどのチームが強いと思う？"
]

baseline_responses = {}

print("--- ベースライン評価開始 ---")
for i, q_nba in enumerate(questions):
    print(f"\n質問 {i+1}: {q_nba}")
    response = generate_output(q_nba)
    print(f"LLMの回答: {response}")
    baseline_responses[q_nba] = response

print("\n--- ベースライン評価終了 ---")

print("\n--- ベースライン評価結果一覧 ---")
for q, r in baseline_responses.items():
    print(f"質問: {q}")
    print(f"回答: {r}\n")


--- ベースライン評価開始 ---

質問 1: NBAって何？
LLMの回答: NBAは、**National Basketball Association (アメリカンバスケットボール協会)** の略称です。 

簡単に言うと、**アメリカで人気のあるバスケットボールリーグ** です。 

**主な特徴:**

* **世界的に人気のあるスポーツ:**  NBAは世界中で人気があり、多くのファンがいます。
* **トップレベルの選手:**  NBAには、世界的に有名な選手が所属しています。
* **注目度の高い試合:**  NBAの試合は、テレビやインターネットで世界中から観戦できます。
* **チームの競争:**  各チームは、シーズンを通して、他のチームと競い合います。


 


 


質問 2: 2023-2024シーズンのNBAのウェスタンカンファレンスの最高勝率チームとその戦績は？
LLMの回答:  申し訳ありませんが、リアルタイムの情報にアクセスできないため、2023-2024シーズンにおけるウェスタンカンファレンスの最高勝率チームとその戦績をお伝えできません。

最新のNBA情報を取得するには、信頼できるスポーツニュースサイトをご確認ください。 

例えば、以下のサイトが参考になります。

* **NBA公式サイト:** https://www.nba.com/
* **ESPN:** https://www.espn.com/nba/
* **Yahoo!スポーツ:** https://sports.yahoo.com/nba/



 


質問 3: 有名なNBA選手を5人教えて
LLMの回答: 有名なNBA選手を５人紹介します！

1. **LeBron James:**  NBAのレジェンドとして知られる、非常に才能と強さを持つ選手。
2. **Stephen Curry:**  3ポイントの技術が非常に高い、人気のある選手。
3. **Giannis Antetokounmpo:**  身体能力とパワーで、チームに大きな影響を与える選手。
4. **Kevin Durant:**  高いシュート力と、チームに安定をもたらす選手。
5. **Nikola Jokic:**  パスと得点力、そしてチームのリーダーシップを持つ選手。



- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 今回はgemmaでは性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

In [7]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
# 1. CSVファイルのパスリスト　適宜変更してください
csv_file_paths = [
    "/content/lecture-ai-engineering/day3/nba_data/nba_2024_player_per_game_stats.csv",
    "/content/lecture-ai-engineering/day3/nba_data/nba_2024_standings_east.csv",
    "/content/lecture-ai-engineering/day3/nba_data/nba_2024_standings_west.csv",
    "/content/lecture-ai-engineering/day3/nba_data/nba_2024_team_per_game_stats.csv"
]


# with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4_raw.txt", "r") as f:
#   raw_writedown = f.read()

In [ ]:
# 2. ドキュメント（テキストチャンク）の作成
documents = []

# 2-1. Player Per Game Stats からドキュメント作成
# 2-1. Player Per Game Stats からドキュメント作成
try:
    df_player_pg = pd.read_csv(csv_file_paths[0])
    # Player列が'Player'である行（ヘッダーの繰り返し）を除外
    # また、Teamが 'nTM' (例: '2TM', '3TM') の行も除外する
    df_player_pg = df_player_pg[
        (df_player_pg['Player'] != 'Player') &
        (~df_player_pg['Team'].astype(str).str.contains(r'\dTM', na=False)) # '2TM', '3TM' などを除外
    ].copy()
    df_player_pg.reset_index(drop=True, inplace=True)

    for index, row in df_player_pg.iterrows():
        player_name = row.get('Player', '不明な選手')
        age = row.get('Age', '不明な年齢')
        team_name = row.get('Team', '不明なチーム')
        pos = row.get('Pos', '不明なポジション')
        g = row.get('G', '不明な試合数')
        gs = row.get('GS', '不明な先発試合数')
        mp = row.get('MP', '不明な平均出場時間')

        # オフェンススタッツ
        fg = row.get('FG', '不明')
        fga = row.get('FGA', '不明')
        fg_pct = row.get('FG%', '不明')
        three_p = row.get('3P', '不明')
        three_pa = row.get('3PA', '不明')
        three_p_pct = row.get('3P%', '不明')
        two_p = row.get('2P', '不明')
        two_pa = row.get('2PA', '不明')
        two_p_pct = row.get('2P%', '不明')
        efg_pct = row.get('eFG%', '不明')
        ft = row.get('FT', '不明')
        fta = row.get('FTA', '不明')
        ft_pct = row.get('FT%', '不明')

        orb = row.get('ORB', '不明') # Offensive Rebounds
        drb = row.get('DRB', '不明') # Defensive Rebounds
        trb = row.get('TRB', '不明') # Total Rebounds
        ast = row.get('AST', '不明') # Assists
        stl = row.get('STL', '不明') # Steals
        blk = row.get('BLK', '不明') # Blocks
        tov = row.get('TOV', '不明') # Turnovers
        pf = row.get('PF', '不明') # Personal Fouls
        pts = row.get('PTS', '不明') # Points

        awards = str(row.get('Awards', '')).strip()

        doc_text_parts = [
            f"{player_name}選手 ({age}歳、{pos}) は2023-2024シーズンに{team_name}に所属し、{g}試合に出場（うち{gs}試合先発）、1試合平均{mp}分プレイしました。",
            f"平均スタッツ: {pts}得点、{trb}リバウンド、{ast}アシスト、{stl}スティール、{blk}ブロック。",
            f"フィールドゴール: 成功{fg}本/試投{fga}本 (成功率{fg_pct})。",
            f"3ポイントシュート: 成功{three_p}本/試投{three_pa}本 (成功率{three_p_pct})。",
            f"2ポイントシュート: 成功{two_p}本/試投{two_pa}本 (成功率{two_p_pct})。",
            f"フリースロー: 成功{ft}本/試投{fta}本 (成功率{ft_pct})。",
            f"ターンオーバーは平均{tov}回、パーソナルファウルは平均{pf}回でした。"
        ]
        if awards and awards != "nan":
            doc_text_parts.append(f"受賞歴: {awards}。")

        doc_text = " ".join(doc_text_parts)
        documents.append(doc_text.strip())

except FileNotFoundError:
    print(f"ファイルが見つかりません: {csv_file_paths[0]}")
except Exception as e:
    print(f"エラーが発生しました ({csv_file_paths[0]}): {e}")


# 2-2. Standings East
try:
    df_standings_east = pd.read_csv(csv_file_paths[1])
    if not df_standings_east.empty:
        team_name_col_east = df_standings_east.columns[0]
        for index, row in df_standings_east.iterrows():
            team_name_raw = row.get(team_name_col_east)
            if pd.isna(team_name_raw) or team_name_raw == team_name_col_east or team_name_raw == "Team": # "Team"というヘッダーも考慮
                continue
            team_name = str(team_name_raw).replace('*', '')
            wins = row.get('W', '不明な勝利数')
            losses = row.get('L', '不明な敗戦数')
            win_loss_pct = row.get('W/L%', '不明な勝率')

            doc_text = (
                f"2023-2024シーズン NBAイースタンカンファレンスにおいて、{team_name}は"
                f"{wins}勝{losses}敗、勝率{win_loss_pct}でした。"
            )
            documents.append(doc_text.strip())
except FileNotFoundError:
    print(f"ファイルが見つかりません: {csv_file_paths[1]}")
except Exception as e:
    print(f"エラーが発生しました ({csv_file_paths[1]}): {e}")

# 2-3. Standings West
try:
    df_standings_west = pd.read_csv(csv_file_paths[2])
    if not df_standings_west.empty:
        team_name_col_west = df_standings_west.columns[0]
        for index, row in df_standings_west.iterrows():
            team_name_raw = row.get(team_name_col_west)
            if pd.isna(team_name_raw) or team_name_raw == team_name_col_west or team_name_raw == "Team":
                continue
            team_name = str(team_name_raw).replace('*', '')
            wins = row.get('W', '不明な勝利数')
            losses = row.get('L', '不明な敗戦数')
            win_loss_pct = row.get('W/L%', '不明な勝率')

            doc_text = (
                f"2023-2024シーズン NBAウェスタンカンファレンスにおいて、{team_name}は"
                f"{wins}勝{losses}敗、勝率{win_loss_pct}でした。"
            )
            documents.append(doc_text.strip())
except FileNotFoundError:
    print(f"ファイルが見つかりません: {csv_file_paths[2]}")
except Exception as e:
    print(f"エラーが発生しました ({csv_file_paths[2]}): {e}")


# 2-4. Team Per Game Stats
try:
    df_team_pg = pd.read_csv(csv_file_paths[3])
    df_team_pg = df_team_pg[df_team_pg['Team'] != 'Team'].copy()
    df_team_pg.reset_index(drop=True, inplace=True)

    for index, row in df_team_pg.iterrows():
        team_name_raw = row.get('Team')
        if pd.isna(team_name_raw):
            continue
        team_name = str(team_name_raw).replace('*', '')
        pts_pg = row.get('PTS', '不明な平均得点')
        opp_pts_pg = row.get('PA/G', '不明な平均失点')
        fg_pct = row.get('FG%', '不明なFG%')

        doc_text = (
            f"2023-2024シーズン、NBAチームの{team_name}は1試合平均{pts_pg}得点、"
            f"平均失点は{opp_pts_pg}、フィールドゴール成功率は{round(float(fg_pct)*100, 1) if isinstance(fg_pct, (float, int)) or (isinstance(fg_pct, str) and fg_pct.replace('.','',1).isdigit()) else '不明なFG%'}%でした。"
        )
        documents.append(doc_text.strip())
except FileNotFoundError:
    print(f"ファイルが見つかりません: {csv_file_paths[3]}")
except Exception as e:
    print(f"エラーが発生しました ({csv_file_paths[3]}): {e}")



In [10]:
print(f"\n--- ドキュメント作成結果の確認 ---")
print(f"作成されたドキュメント総数: {len(documents)}")

if documents:
    print("\n最初の数件のドキュメント例:")
    for i, doc in enumerate(documents[:5]): # 最初の5件を表示
        print(f"ドキュメント {i+1}: {doc}")

    if len(documents) > 10:
        print("\n途中のドキュメント例 (100件目あたりから):")
        start_index = min(100, len(documents) - 3)
        for i, doc in enumerate(documents[start_index : start_index + 3]):
             print(f"ドキュメント {start_index + i + 1}: {doc}")

    print("\n最後の数件のドキュメント例:")
    for i, doc in enumerate(documents[-3:]):
        print(f"ドキュメント {len(documents) - 3 + i + 1}: {doc}")
else:
    print("documentsリストは空です。")
print("-" * 50)


--- ドキュメント作成結果の確認 ---
作成されたドキュメント総数: 719

最初の数件のドキュメント例:
ドキュメント 1: Joel Embiid選手 (29.0歳、C) は2023-2024シーズンにPHIに所属し、39.0試合に出場（うち39.0試合先発）、1試合平均33.6分プレイしました。 平均スタッツ: 34.7得点、11.0リバウンド、5.6アシスト、1.2スティール、1.7ブロック。 フィールドゴール: 成功11.5本/試投21.8本 (成功率0.529)。 3ポイントシュート: 成功1.4本/試投3.6本 (成功率0.388)。 2ポイントシュート: 成功10.2本/試投18.3本 (成功率0.556)。 フリースロー: 成功10.2本/試投11.6本 (成功率0.883)。 ターンオーバーは平均3.8回、パーソナルファウルは平均2.9回でした。 受賞歴: AS。
ドキュメント 2: Luka DonÄiÄ選手 (24.0歳、PG) は2023-2024シーズンにDALに所属し、70.0試合に出場（うち70.0試合先発）、1試合平均37.5分プレイしました。 平均スタッツ: 33.9得点、9.2リバウンド、9.8アシスト、1.4スティール、0.5ブロック。 フィールドゴール: 成功11.5本/試投23.6本 (成功率0.487)。 3ポイントシュート: 成功4.1本/試投10.6本 (成功率0.382)。 2ポイントシュート: 成功7.4本/試投13.0本 (成功率0.573)。 フリースロー: 成功6.8本/試投8.7本 (成功率0.786)。 ターンオーバーは平均4.0回、パーソナルファウルは平均2.1回でした。 受賞歴: MVP-3,CPOY-6,AS,NBA1。
ドキュメント 3: Giannis Antetokounmpo選手 (29.0歳、PF) は2023-2024シーズンにMILに所属し、73.0試合に出場（うち73.0試合先発）、1試合平均35.2分プレイしました。 平均スタッツ: 30.4得点、11.5リバウンド、6.5アシスト、1.2スティール、1.1ブロック。 フィールドゴール: 成功11.5本/試投18.8本 (成功率0.611)。 3ポイントシュート: 成功0.5本/試投1.7本 (成功率0.274)。 2

In [17]:
# references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
# query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
# response = generate_output(query)
# print(response)

# 3. エンベディングと検索 (ここからは提供されたコードとほぼ同じ)
# 質問リストを定義
questions_nba = [
    # "有名なNBA選手を5人教えて",
     "2023-2024シーズンのAll NBA First teamに選ばれた5人は？",
    # "2024-2025のシーズンのNBAはどのチームが強いと思う？"
]

# 各質問についてRAGを実行
rag_responses = {}
if not documents:
    print("skip")

else:
    print("\n--- RAG評価開始 ---")
    print("ドキュメントのエンベディングを作成中...")
    document_embeddings = emb_model.encode(documents)
    print("ドキュメントのエンベディング作成完了。")

    for i_q, question_nba in enumerate(questions_nba):
        print(f"\nRAG 質問 {i_q+1}: {question_nba}")

        query_embeddings = emb_model.encode([question_nba], prompt_name="query")
        scores = (query_embeddings @ document_embeddings.T)
        topk = 20

        rag_query_prompt = question_nba

        if scores is not None and scores.shape[1] > 0:
            sorted_indices = scores.argsort()[0][::-1][:min(topk, scores.shape[1])]

            print(f"取得した上位{len(sorted_indices)}件のドキュメント:")
            retrieved_docs_for_prompt = []
            for rank, index in enumerate(sorted_indices):
                print(f"  ドキュメント{rank+1} (Score: {scores[0][index]:.4f}):")
                print(f"    {documents[index]}\n")
                retrieved_docs_for_prompt.append(documents[index])

            if retrieved_docs_for_prompt:
                references_for_prompt = "\n".join(["* " + doc for doc in retrieved_docs_for_prompt])
                rag_query_prompt = f"[参考資料]\n{references_for_prompt}\n\n[質問] {question_nba}"
            else:
                print("関連性の高いドキュメントが見つかりませんでした")
        else:
            print("スコア計算に問題があるか、ドキュメントがありません。質問のみで回答します。")

        print(f"\nLLMへの最終プロンプト:\n---\n{rag_query_prompt}\n---")
        response_rag = generate_output(rag_query_prompt)
        print(f"RAGあり LLMの回答: {response_rag}")
        rag_responses[question_nba] = response_rag

    print("\n--- RAG評価終了 ---")

    print("\n--- RAG評価結果一覧 ---")
    for q, r in rag_responses.items():
        print(f"質問: {q}")
        print(f"回答: {r}\n")


--- RAG評価開始 ---
ドキュメントのエンベディングを作成中...
ドキュメントのエンベディング作成完了。

RAG 質問 1: 2023-2024シーズンのAll NBA First teamに選ばれた5人は？
取得した上位20件のドキュメント:
  ドキュメント1 (Score: 0.7608):
    LeBron James選手 (39.0歳、PF) は2023-2024シーズンにLALに所属し、71.0試合に出場（うち71.0試合先発）、1試合平均35.3分プレイしました。 平均スタッツ: 25.7得点、7.3リバウンド、8.3アシスト、1.3スティール、0.5ブロック。 フィールドゴール: 成功9.6本/試投17.9本 (成功率0.54)。 3ポイントシュート: 成功2.1本/試投5.1本 (成功率0.41)。 2ポイントシュート: 成功7.5本/試投12.8本 (成功率0.592)。 フリースロー: 成功4.3本/試投5.7本 (成功率0.75)。 ターンオーバーは平均3.5回、パーソナルファウルは平均1.1回でした。 受賞歴: CPOY-10,AS,NBA3。

  ドキュメント2 (Score: 0.7606):
    Anthony Davis選手 (30.0歳、C) は2023-2024シーズンにLALに所属し、76.0試合に出場（うち76.0試合先発）、1試合平均35.5分プレイしました。 平均スタッツ: 24.7得点、12.6リバウンド、3.5アシスト、1.2スティール、2.3ブロック。 フィールドゴール: 成功9.4本/試投16.9本 (成功率0.556)。 3ポイントシュート: 成功0.4本/試投1.4本 (成功率0.271)。 2ポイントシュート: 成功9.0本/試投15.5本 (成功率0.582)。 フリースロー: 成功5.5本/試投6.8本 (成功率0.816)。 ターンオーバーは平均2.1回、パーソナルファウルは平均2.3回でした。 受賞歴: DPOY-4,AS,NBA2,DEF1。

  ドキュメント3 (Score: 0.7605):
    Jayson Tatum選手 (25.0歳、PF) は2023-2024シーズンにBOSに所属し、74.0試合に出場（うち74.0試合先発）、1試合平均35.7分プレイし